In [1]:
import sys
sys.path.insert(0, '/home/niell_lab/Documents/github/FreelyMovingEphys/')
import warnings
warnings.filterwarnings('ignore')
from utils.object_avoidance import AvoidanceSession

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
# metadata json path
metadata_path = '/home/niell_lab/data/object_avoidance/gap_detection_recordings/gd_metadata.json'
# task name
# 'oa' for object avoidance
# 'gd' for gap detection
task_name = 'gd'

In [3]:
# create the session
session = AvoidanceSession(metadata_path, task=task_name)

In [4]:
# pose estimation with DLC
# you can skip this, if the .h5 files already exist

# session.preprocess()

In [5]:
# run the complete analysis
# saves out an .h5 and .pdf for each recording in the metadata .json
session.process()

3it [01:15, 25.07s/it]


ValueError: All-NaN slice encountered

In [ ]:
# everything below here is reading and collecting all data in a single dataframe
# but will only include recordings listed in the metadata.json

In [ ]:
import os
import pandas as pd
from utils.aux_funcs import find

In [ ]:
session.gather_all_sessions()

In [ ]:
recordings = []
for date in session.dates_list:
    date_dir = os.path.join(session.path, date)
    for animal in [i for i in list(session.metadata[date].keys())]:
        animal_dir = os.path.join(date_dir, animal)
        camconfig = session.generic_camconfig
        camconfig['animal_directory'] = animal_dir
        for recording_name in [k for k,v in session.metadata[date][animal].items()]:
            recording_dir = os.path.join(animal_dir, recording_name)
            recordings.append(recording_dir)

In [ ]:
conditions = ['gd','gddark','gdwhsk','gdwhskdark']

In [ ]:
allgd = pd.DataFrame([])
for r in recordings:
    file = [i for i in find('*.h5',r) if 'DLC' not in i][0]
    data = pd.read_hdf(file)
    allgd = pd.concat([allgd, data], axis=0)

In [ ]:
allgd.columns.values